In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(tot_pages):
    for page in range(51,tot_pages+1, 51):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f'https://beg-dorozhki.ru/category/offset{page}'
        response = requests.get(url_main, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'ui-link')
        for item in data:
            if bool(re.search(pattern = re.escape('/product/'), string = item['href'])):
                item_url = 'https://beg-dorozhki.ru' + item['href']
                yield item_url

In [3]:
def get_item(tot_pages):

    for url_item in get_url(tot_pages):
        response = requests.get(url_item, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
            title = title.replace('\r', '').replace('\t', '').replace('\n', '').strip()
        except:
            title = ''

        try:
            article = soup.find(class_ = 'b-goods__article').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '').strip()
        except:
            article = ''

        try:
            image = soup.find(class_ = 'img-scale', src = True)
            url = 'https://beg-dorozhki.ru' + image['src']

        except:
            print(f'Log: no image for {article} found...')
            url = ''

        try:
            char = soup.find_all(class_ = 'b-goods-specif__item param')
            char_string = ''
            for ch in char:
                children = ch.children
                for child in children:
                    char_string += child.text
                    char_string += ' '
            chars = char_string.strip()
        except:
            chars = ''

        try:
            desc = soup.find(class_ = 'b-goods-info')
            desc = desc.text.replace('\n', ' ').strip()
        except:
            desc = ''

        try:
            price = soup.find(class_ = 'b-goods-price__new').text
            price = price.replace(' ', '').replace('руб.', '')
        except:
            price =''

        try:
            cat = soup.find(class_ = 'breadcrumb').text
            cat_lst = []
            for i in cat.split('\n'):
                if i != '':
                    cat_lst.append(i)
        except:
            cat_lst = []
        try:
            cat_1 = cat_lst[0].strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat_lst[1].strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat_lst[2].strip()
        except:
            cat_3 = ''

        yield  title, article, price, cat_1, cat_2, cat_3, url, desc, chars

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics']

df = pd.DataFrame(columns = df_columns)
save_frequency = 100
log_frequency = 300

In [5]:
def to_csv(tot_pages):
    counter = 0  

    for item in get_item(tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('beg_dorozh.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('beg_dorozh.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv(663)

100 items saved in csv..
200 items saved in csv..
300 items saved in csv..
400 items saved in csv..
500 items saved in csv..
600 items saved in csv..

Total items saved: 615


In [7]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=10000></img></a>'.format(val, val)
df.iloc[44:46].style.format({'url': show_image, **{'width': '100px'}})

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
44,Беговая дорожка Itosima Crown IT510,9208,39990,Беговые дорожки,ITOSIMA,Itosima Crown IT510,,"Описание Itosima Crown IT510 Itosima Crown IT510 Система шумоподавления ITOSIMA обеспечивает низкий уровень шума при мощном двигателе 1.5 л.с., развивающим скорость до 12 км/ч. Тренажер подходит для пользователей весом до 110 кг, 16 программ тренировок помогут выбрать наиболее подходящий уровень занятий. Одним их отличий тренажера является система складывания «SuperFold», позволяющая компактно сложить тренажер под кровать, за дверь, либо в шкаф. Благодаря системе смазывания бегового полотна «Easy Oil» имеется возможность самостоятельно за ним ухаживать. Быстрые кнопки старт/стоп и регулировки скорости на поручнях обеспечивают необходимые удобства пользователям. Во время тренировки можно подключить смартфон по Bluetooth (iOs/Android), используя приложение GFIT, также имеется держатель для планшета. Дополнительно встроен MP3 проигрыватель со встроенной акустикой.","Тип электрическая Уровень базовый Габариты (ДхШхВ) 146 х 67 х 117 cм Вес тренажера 40 кг Вес в упаковке 50 кг Возможность складывания есть, система Superfold Габариты в сложенном виде 32 х 67 х 146 см Тип питания сеть 220 Вольт Происхождение марки Япония Страна-производитель Китай Гарантия на раму 5 лет Гарантия 2 года Максимальная нагрузка 110 кг Двигатель 1.5 л. с. Максимальная скорость 12 км/ч Максимальный вес пользователя 110 кг Беговое полотно двухслойное DIAMOND Размеры полотна (ДхШ) 123 х 41 см Толщина деки 16 мм Количество программ 16 Предустановленные программы 12 Пользовательские программы 1 Измерение пульса сенсоры на рукоятках Мультимедиа есть,встроенная стереоакустика Дисплей LCD 5 дюймов с подсветкой Отображение времени тренировки есть Отображение пройденного расстояния есть Отображение пульса есть Отображение расхода калорий есть Отображение текущей скорости есть Отображение профиля программы есть Габариты в упаковке 156 х 76 х 27.5 cм Система складывания ""SuperFold"" Транспортировочные ролики есть Дополнительно Держатель под планшет. Система смазки ""EasyOil"". Возможность тренировок с помощью смартфона(iOs/Android) через Bluetooth Вес нетто 40 кг"
45,Беговая дорожка Titanium One T42 S,9378,39990,Беговые дорожки,Titanium,Titanium One T42 S,,"Описание Titanium One T42 S Скачать инструкцию от Titanium One T42 S Плоская беговая дорожка — это популярный вид кардиотренажёра для домашних тренировок в условиях ограниченного пространства. Т. к. в таких дорожках нет встроенного кардиодатчика, то пульс и прочие индивидуальные показатели можно измерять с помощью фитнес-браслета. Простота в обращении Дорожку очень легко настроить для занятий, она не требует сборки. Достаточно достать из коробки, подключить к электросети, и она готова к использованию. Управление скоростью происходит с пульта дистанционного управления. Также на нём отображаются основные показатели тренировки: время, пройденная дистанция, текущая скорость, затраченные калории. Пульт имеет ремешок для крепления на запястье, поэтому не придётся отвлекаться от тренировки для переключения скорости. Компактность и мобильность Тренажёр имеет очень малые размеры (145 см в длину и 61 см в ширину), благодаря чему заниматься можно в даже в небольшом помещении. Толщина у One T42 S всего 14 см, поэтому найти место для хранения дорожки в свободное от тренировок время не составит труда — под кроватью, за дверью, у стены, в нише, на балконе. Простоту перемещения обеспечивают транспортировочные ролики и малый вес всей конструкции (19 кг). Функциональные возможности Антискользящее беговое полотно имеет комфортные размеры (122 см в длину и 43 см в ширину), что подходит пользователям ростом до 180 см. В основе находится парафинированная дека толщиной 16 мм. Инженеры Titanium использовали разработку компании Leeson Electric — двигатель мощностью 1,75 л.с., который обеспечивает плавные движения полотна без рывков, а также разгоняет его до 10 км/ч. Такого за

In [8]:
df

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
0,Беговая дорожка CardioPower T25,9201,44900,Беговые дорожки,CardioPower,CardioPower T25,https://beg-dorozhki.ru/published/publicdata/B...,Описание CardioPower T25 Cardiopower T25 – д...,Тип электрическая Уровень базовый Габариты (Дх...
1,Беговая дорожка SVENSSON BODY LABS BALANCE A,9944,48990,Беговые дорожки,Svensson Body Labs,SVENSSON BODY LABS BALANCE A,https://beg-dorozhki.ru/published/publicdata/B...,Описание SVENSSON BODY LABS BALANCE A Скача...,Тип электрическая Уровень базовая Габариты (Дх...
2,Беговая дорожка SVENSSON BODY LABS BALANCE B,9946,54890,Беговые дорожки,Svensson Body Labs,SVENSSON BODY LABS BALANCE B,https://beg-dorozhki.ru/published/publicdata/B...,Описание SVENSSON BODY LABS BALANCE B Беговая...,Тип электрическая Уровень базовая Габариты (Дх...
3,Беговая дорожка Laufstein Corsa Automatik,9173,56990,Беговые дорожки,Laufstein,Laufstein Corsa Automatik,https://beg-dorozhki.ru/published/publicdata/B...,Описание Laufstein Corsa Automatik Laufstein ...,Тип электрическая Габариты (ДхШхВ) 160 х 70 х ...
4,Беговая дорожка Titanium Masters Physiotech TBM,9245,56990,Беговые дорожки,Titanium,Titanium Masters Physiotech TBM,https://beg-dorozhki.ru/published/publicdata/B...,Описание Titanium Masters Physiotech TBM Инно...,Тип электрическая Габариты (ДхШхВ) 152 х 76 х ...
...,...,...,...,...,...,...,...,...,...
610,Беговая дорожка Vision Fitness T9700 HRT,1338,159890,Беговые дорожки,Vision,Vision Fitness T9700 HRT,https://beg-dorozhki.ru/published/publicdata/B...,Описание Vision Fitness T9700 HRT Техническое...,Тип электрическая Уровень профессионал Габарит...
611,Беговая дорожка Johnson T7000 PRO,9023,159890,Беговые дорожки,Johnson,Johnson T7000 PRO,https://beg-dorozhki.ru/published/publicdata/B...,Описание Johnson T7000 PRO Johnson T7000 PRO...,Тип электрическая Уровень профессионал Габарит...
612,Беговая дорожка Vision Fitness T9800 HRT,1339,169890,Беговые дорожки,Vision,Vision Fitness T9800 HRT,https://beg-dorozhki.ru/published/publicdata/B...,Описание Vision Fitness T9800 HRT Техническое...,Тип электрическая Уровень профессиональный Габ...
613,Беговая дорожка Body Solid T10HRC,1545,199900,Беговые дорожки,Body Solid,Body Solid T10HRC,https://beg-dorozhki.ru/published/publicdata/B...,Описание Body Solid T10HRC Техническое описан...,Тип электрическая\t Уровень профессиональная Г...


In [10]:
df.iloc[613:615].style.format({'url': show_image, **{'width': '100px'}})

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
613,Беговая дорожка Body Solid T10HRC,1545,199900,Беговые дорожки,Body Solid,Body Solid T10HRC,,"Описание Body Solid T10HRC Техническое описание Body Solid T10HRC: Модель прекрасно подходит для домашних тренировок и коммерческой эксплуатации. Мощный двигатель 3.0 л.с. совместно с системой ограждающей двигатель от короткого замыкания обеспечивают долговечную работу беговой дорожки. Прочная стальная конструкция рамы рассчитана на максимальный вес пользователя до 159 кг. Это говорит о высокой надежности тренажера и позволяет заниматься на нем ежедневно без угрозы поломок. Профессиональное беговое полотно размером 152 х 51 см позволяет пользователям с любым ростом и комплекцией заниматься комфортно и безопасно. Амортизационная система представлена в виде подвеса деки в 6 точках снижает нагрузки на суставы и позвоночник пользователя. Максимальная скорость 16 км/ч позволяет заниматься в быстром темпе бега. Изменяемый угол наклона от 0 до 15° позволяет разнообразить тренировки и максимально проработать все группы мышц. Для более эффективной и приятной тренировки дорожка оснащена электронной регулировкой наклона с помощью простого нажатия клавиш. Тренажер не обладает системой складывания, что лишний раз подтверждает прочность конструкции. У тренажера есть встроенные подставки для бутылок с водой и мелких вещей. Особенности программного пакета: Консоль беговой дорожки матричным ЖК дисплеем, который отображает: пульс, скорость, угол наклона, время, калории, дистанцию, статистику. Тренировочный компьютер имеет 12 программ для разнообразных тренировок: выносливость, бег, интервальная нагрузка, прогулочная, произвольная, дистанция, калории, время, ручная, сжигание жира, тренировка сердечной мышцы и 1 пульсозависимая. Измерение пульса во время тренировки осуществляется беспроводным нагрудным кардиодатчиком. Вместе с пульсозависимой программой получается полноценный кардиоцентр для тренировки сердечнососудистой системы. Для удобства пользователя частоиспользуемые клавиши расположены рядом с рукоятками тренажера. Возможность подключения MP3/CD позволяют тренироваться под любимую музыку. Особенности по сравнению с другими тренажерами в этой ценовой категории: Нагрудный кардиодатчик (в комплекте) обеспечивает точное измерение пульса, а вместе с пульсозависимыми программами получается полноценный кардиокомплекс. Благодаря своим характеристикам тренажер адаптирован для использования в фитнес-центрах.","Тип электрическая Уровень профессиональная Габариты (ДхШхВ) 196 х 81 х 145 см Возможность складывания отсутствует Тип питания сеть 220 Вольт Потребляемая мощность 1.8 кВт/ч Происхождение марки США Страна-производитель Тайвань Гарантия 1 год Количество программ 12 Пульсозависимые программы 1 Спецификации программ выносливость, бег, интервальная нагрузка, прогулочная, произвольная, дистанция, калории, время, ручная, сжигание жира, тренировка сердечной мышцы Измерение пульса Polar приемник Мультимедиа CD/MP3 Двигатель 3.0 л.с Максимальная скорость 16 км/ч Максимальный вес пользователя 159 кг Размеры полотна (ДхШ) 152 х 51 см Толщина деки 16 мм Возможный наклон от 0 до 15° Регулировка наклона электронная Амортизация подвес деки в 6 точках Дисплей матричный ЖК Отображение времени тренировки есть Отображение пройденного расстояния есть Отображение пульса есть Отображение расхода калорий есть Отображение текущей скорости есть Отображение профиля программы есть Рама стальная Дополнительно отсек для бутылки"
614,Беговая дорожка AeroFit PRO 8800TM LCD,1188,200000,Беговые дорожки,AeroFIT,AeroFit PRO 8800TM LCD,,"Описание AeroFit PRO 8800TM LCD AeroFit PRO 8800TM LCDAeroFit PRO 8800TM LCD — современная многофункциональная дорожка, идеально подходящая для домашнего и коммерческого использования (в бюджетных фитнес-залах, гостиницах и т. д.). Она поможет избавиться от лишних килограммов, укрепить здоровье и наполниться энергией.Главные преимуществаОсновными достоинствами этой модели являются: 